In [ ]:
#Load packages

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.model_selection as mod
import sklearn.neighbors as nei
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score 
from sklearn.naive_bayes import MultinomialNB
# from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import operator
%matplotlib inline

In [ ]:
url="https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv"

In [ ]:
#Load CSV file using Pandas

pd = pd.read_csv(url)

In [ ]:
data = pd.copy()

In [ ]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
data = data.drop(['DiabetesPedigreeFunction', 'Pregnancies', 'Insulin', 'SkinThickness'], axis=1)

In [ ]:
f = ['Glucose', 'BloodPressure', 'BMI', 'Age']

for _ in f:
    print(_,"= ",(data[_]==0).sum())  

Glucose =  5
BloodPressure =  35
BMI =  11
Age =  0


In [ ]:
#missing value fillup
data['BMI']=data.BMI.mask(data.BMI == 0,(data['BMI'].mean(skipna=True)))
# data['SkinThickness']=data.SkinThickness.mask(data.SkinThickness == 0,(data['SkinThickness'].mean(skipna=True)))
data['BloodPressure']=data.BloodPressure.mask(data.BloodPressure == 0,(data['BloodPressure'].mean(skipna=True)))
data['Glucose']=data.Glucose.mask(data.Glucose == 0,(data['Glucose'].mean(skipna=True)))
print(data.head(15))

    Glucose  BloodPressure        BMI  Age  Outcome
0     148.0      72.000000  33.600000   50        1
1      85.0      66.000000  26.600000   31        0
2     183.0      64.000000  23.300000   32        1
3      89.0      66.000000  28.100000   21        0
4     137.0      40.000000  43.100000   33        1
5     116.0      74.000000  25.600000   30        0
6      78.0      50.000000  31.000000   26        1
7     115.0      69.105469  35.300000   29        0
8     197.0      70.000000  30.500000   53        1
9     125.0      96.000000  31.992578   54        1
10    110.0      92.000000  37.600000   30        0
11    168.0      74.000000  38.000000   34        1
12    139.0      80.000000  27.100000   57        0
13    189.0      60.000000  30.100000   59        1
14    166.0      72.000000  25.800000   51        1


In [ ]:
### preprocessing

data['Age']=data['Age'].astype(int)
data.loc[data['Age'] <= 16, 'Age']= 0
data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
data.loc[data['Age'] > 64, 'Age'] = 4

data['Glucose']=data['Glucose'].astype(int)
data.loc[data['Glucose'] <= 80, 'Glucose']= 0
data.loc[(data['Glucose'] > 80) & (data['Glucose'] <= 100), 'Glucose'] = 1
data.loc[(data['Glucose'] > 100) & (data['Glucose'] <= 125), 'Glucose'] = 2
data.loc[(data['Glucose'] > 125) & (data['Glucose'] <= 150), 'Glucose'] = 3
data.loc[data['Glucose'] > 150, 'Glucose'] = 4

data['BloodPressure']=data['BloodPressure'].astype(int)
data.loc[data['BloodPressure'] <= 50, 'BloodPressure']= 0
data.loc[(data['BloodPressure'] > 50) & (data['BloodPressure'] <= 65), 'BloodPressure'] = 1
data.loc[(data['BloodPressure'] > 65) & (data['BloodPressure'] <= 80), 'BloodPressure'] = 2
data.loc[(data['BloodPressure'] > 80) & (data['BloodPressure'] <= 100), 'BloodPressure'] = 3
data.loc[data['BloodPressure'] > 100, 'BloodPressure'] = 4

In [ ]:
X = data.iloc[: , :-1].values
y = data.iloc[: , -1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X, y , test_size = 0.2,random_state=0 )

In [ ]:
X_train

array([[ 3.        ,  2.        , 35.2       ,  3.        ],
       [ 1.        ,  1.        , 28.2       ,  1.        ],
       [ 4.        ,  3.        , 52.3       ,  1.        ],
       ...,
       [ 1.        ,  1.        , 24.7       ,  1.        ],
       [ 1.        ,  2.        , 30.1       ,  2.        ],
       [ 3.        ,  3.        , 31.99257812,  4.        ]])

In [ ]:
X_train[0], X_test[0], X_test[1]

(array([ 3. ,  2. , 35.2,  3. ]),
 array([ 4. ,  2. , 42.9,  1. ]),
 array([ 2. ,  2. , 33.6,  1. ]))

In [ ]:
# another preprocessing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc1 = StandardScaler()

# sc.fit(X_train)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
sc.mean_, sc.scale_

(array([ 2.23778502,  1.93322476, 32.62351893,  1.5732899 ]),
 array([1.14806407, 0.79724915, 6.98896781, 0.77438844]))

In [ ]:
X_train[0], X_test[0]

(array([0.66391328, 0.08375706, 0.36864973, 1.8423701 ]),
 array([ 1.5349448 ,  0.08375706,  1.47038609, -0.7403131 ]))

In [ ]:
(1-1.5732899)/0.77438844

-0.7403130914505904

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
q = []
i=[3,4,5,6]
j=[3,4,5,6]

for a in i:
    for b in j:
        #initialising the ANN
        classifier = Sequential()

        #adding the input layer and the first hidden layer
        classifier.add(Dense(activation="relu", input_dim=4, units=a, kernel_initializer="uniform"))

        #adding the second hidden layer
        classifier.add(Dense(b, activation="relu"))

        #adding the output layer
        classifier.add(Dense(1,  activation="sigmoid"))

        #compiling the ANN
        classifier.compile(optimizer = "adam", loss ="binary_crossentropy" , metrics = ['accuracy'])

        #fitting the ANN to the training set
        classifier.fit(X_train,y_train , batch_size = 10 , epochs = 100)
        
        y_pred = classifier.predict(X_test)

        y_pred = (y_pred>0.5)
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test,y_pred)
        accuracy=((cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+cm[0,0]+cm[1,0]))
        q.append(accuracy)
        print(q)

In [ ]:
q

[0.7597402597402597,
 0.7792207792207793,
 0.7792207792207793,
 0.7792207792207793,
 0.7662337662337663,
 0.7922077922077922,
 0.7597402597402597,
 0.7792207792207793,
 0.7727272727272727,
 0.7597402597402597,
 0.7662337662337663,
 0.7987012987012987,
 0.7662337662337663,
 0.7987012987012987,
 0.7662337662337663,
 0.7727272727272727]

In [ ]:
classifier = Sequential()

#adding the input layer and the first hidden layer
classifier.add(Dense(activation="relu", input_dim=4, units=6, kernel_initializer="uniform"))

#adding the second hidden layer
classifier.add(Dense(4, activation="relu"))

#adding the output layer
classifier.add(Dense(1,  activation="sigmoid"))

#compiling the ANN
classifier.compile(optimizer = "adam", loss ="binary_crossentropy" , metrics = ['accuracy'])

#fitting the ANN to the training set
classifier.fit(X_train,y_train , batch_size = 10 , epochs = 500)

y_pred = classifier.predict(X_test)

y_pred = (y_pred>0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
accuracy=((cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+cm[0,0]+cm[1,0]))

In [ ]:
X_test[:2]

array([[-0.89295432,  1.53627706,  0.14230249,  1.80779423,  1.75903114,
        -0.65488952],
       [-0.56553774, -0.22929508,  0.14230249,  0.40980248,  0.290251  ,
        -0.65488952]])

In [ ]:
data.head(1)

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,Age,Outcome
0,6,3,2,35.0,33.6,3,1


In [ ]:
pd.head(1)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1


In [ ]:
accuracy

0.8116883116883117

In [ ]:
from keras.models import load_model

classifier.save('diabetes_model.h5')  # creates a HDF5 file 'my_model.h5'
del classifier  # deletes the existing model

In [ ]:
classifier = load_model('diabetes_model.h5')

In [ ]:
y_pred = classifier.predict(X_test)

y_pred = (y_pred>0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
accuracy=((cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+cm[0,0]+cm[1,0]))


In [ ]:
X_test[0]

NameError: ignored

In [ ]:
X_test[0].reshape(1,4)

array([[ 1.5349448 ,  0.08375706,  1.47038609, -0.7403131 ]])

In [ ]:
(classifier.predict(X_test[1].reshape(1,4)) > 0.5)[0][0]

False

In [ ]:
classifier.predict(X_test[1].reshape(1,4))

array([[0.19641668]], dtype=float32)

# conversion to tflite

In [ ]:
!cp "/content/drive/MyDrive/Projects/Diabetes Prediction/diabetes_model.h5" "/content"

In [ ]:
from keras.models import load_model
classifier = load_model('diabetes_model.h5')

In [ ]:
## conversion to tflite

import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(classifier) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('diabetes_model.tflite', 'wb') as f:
  f.write(tflite_model)

## evaluate tflite

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="diabetes_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

In [ ]:
input_details

[{'name': 'serving_default_dense_57_input:0',
  'index': 0,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([-1,  4], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
type(1.53)sample.astype(np.float32)

float

In [ ]:
input_data = np.array(X_test[1].reshape(1,4), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print((output_data > 0.5)[0][0])

False


In [ ]:
output_data

array([[0.19641668]], dtype=float32)